In [ ]:
## pgeocode to rohan for processing

In [1]:
import pandas as pd
import numpy as np
import re
import pgeocode

In [2]:
master_co_data = pd.read_excel('master_co_data.xlsx', header=0)

working_master = master_co_data.copy()

In [3]:
# Processing to create valid 'pin' file and no 'pin' file

yes_pin_master = working_master[working_master['PIN'].notna()].reset_index(drop=True) # master with some value in PIN
nan_pin_master = working_master[working_master['PIN'].isna()].reset_index(drop=True)  # master with NaN value in PIN

yes_pin_master['PIN'] = yes_pin_master['PIN'].astype(str)                      # converting pin to string

yes_pin_master['PIN'] = yes_pin_master['PIN'].apply(lambda x: re.sub('[^0-9]', '', x))  # 'pins' with only digits
yes_pin_6ch = yes_pin_master.loc[yes_pin_master['PIN'].str.len() == 6]         # pin should be of 6 digits
yes_pin_not_6ch = yes_pin_master.loc[yes_pin_master['PIN'].str.len() != 6]

invalid_pin = yes_pin_6ch[yes_pin_6ch['PIN'].str.startswith(('0'))]            # pins cannot start with '0'
valid_pin = yes_pin_6ch[yes_pin_6ch['PIN'].str.startswith(('1', '2', '3', '4', '5', '6', '7', '8', '9'))]

frames_1 = [invalid_pin, yes_pin_not_6ch]                                      # merging invalid_pin dataframes
merged_invalid_pin = pd.concat(frames_1).reset_index(drop=True)
merged_invalid_pin['PIN'] = ''                                                 # convering 'pin' column to blank and then to NaN
merged_invalid_pin['PIN'] = merged_invalid_pin['PIN'].replace('', np.nan, regex=True)

frames_2 = [nan_pin_master, merged_invalid_pin]                      # merging invalid_pin having pin = Nan with earlier nan_pin
merged_nan_pin = pd.concat(frames_2).reset_index(drop=True)


In [4]:
# pincode extraction from 'no pin' file

merged_nan_pin['ADD'] = merged_nan_pin['ADD'].astype(str).str.lower()       # making all in lower case
merged_nan_pin['PIN'] = merged_nan_pin['PIN'].astype(str)           # making type of column 'PIN' from 'float' to 'str'

merged_nan_pin['PIN'] = merged_nan_pin['ADD'].str.extract(r"\b([1-9]{1}[0-9]{2}\s{0,1}[0-9]{3})\b")  # take out pincode from
                                                      # the address. Here blank pins automatically gets converted to Nan(float)
merged_nan_pin['PIN'] = merged_nan_pin['PIN'].str.replace(' ', '')         # removing gap between digits of pincode
                                                                           # i.e. from 396 191 to 396191

next_nan_pin = merged_nan_pin[merged_nan_pin['PIN'].isna()].reset_index(drop=True)
next_yes_pin = merged_nan_pin[~merged_nan_pin['PIN'].isna()].reset_index(drop=True)

# Here checking for 6 digits and checking for starting with '0' not reqyured since above regex takes care of this

frames_3 = [valid_pin, next_yes_pin]
final_valid_pin = pd.concat(frames_3).reset_index(drop=True)

final_invalid_pin = next_nan_pin


In [ ]:
# Going for geocode :

valid_pin_df = final_valid_pin.copy()
valid_pin_df = valid_pin_df.fillna(0)    # filling all NaN with  zero for subsequent 'if' statement

valid_pin_with_city_state = valid_pin_df[(valid_pin_df[['CITY','STATE']] != 0).all(axis=1)].reset_index(drop=True)
valid_pin_wout_city_state = valid_pin_df[~(valid_pin_df[['CITY','STATE']] != 0).all(axis=1)].reset_index(drop=True)

#========================================================================================================================

geocode_df = valid_pin_wout_city_state.copy()
#geocode_df = zz[3000:3005].reset_index(drop=True)

for i in range(len(geocode_df)):

    nomi = pgeocode.Nominatim('in')
    pin_code = geocode_df['PIN'][i]
    output_data = nomi.query_postal_code(pin_code)

    output_data.replace(np.nan, 0, inplace=True)
    if output_data[1] != 0:
#        city = output_data[2] + ',' + output_data[7] + ',' + output_data[5]
        city = output_data[5]
        state = output_data[3]
        if geocode_df['CITY'][i] == 0:
            geocode_df['CITY'][i] = city
        if geocode_df['STATE'][i] == 0:
            geocode_df['STATE'][i] = state
    
#    print('=======================',i)
#==========================================================================================================================

# If pin is wrong i.e. does not exist, this will not give any result and hence, 'city' or 'state' will remain '0' as 
# original. To take care of this, following two lines

geocode_with_city_state = geocode_df[(geocode_df[['CITY', 'STATE']] != 0).all(axis=1)]
geocode_wout_city_state = nan_pin_with_pin[~(nan_pin_with_pin[['CITY', 'STATE']] != 0).all(axis=1)]

geocode_wout_city_state = ''                                              # convering 'pin' column to blank and then to NaN
geocode_wout_city_state['PIN'] = geocode_wout_city_state['PIN'].replace('', np.nan, regex=True)


frames_4 = [valid_pin_with_city_state, geocode_with_city_state]
df_with_city_pin_state = pd.concat(frames_4).reset_index(drop=True)  ## OUTPUT-1 : master file

frames_5 = [final_invalid_pin, geocode_wout_city_state]
df_with_invalid_pin = pd.concat(frames_5).reset_index(drop=True)     ## OUTPUT-2 : to compare with city_master
